### 1. kakao api 기반으로 최대한 지역 정보 모으기

In [1]:
import requests
import pandas as pd
import time
import re
from datetime import datetime
import os


save_path = '../../data/solar_energy'

# 카카오 API 키 설정
KAKAO_API_KEY = "24dacb9216fed52f413cf2ecb241a4d2"

# 발전소 리스트
power_plants = [
    '감우리', '남제주소내', '무릉리', '부산복합자재창고', '부산수처리장', '부산신항', 
    '부산역선상주차장', '부산운동장', '세화리', '송당리', '신인천 1_2단계 주차장', 
    '신인천 북측부지', '신인천 주차장', '신인천본관주차장', '신인천소내', '신인천전망대', 
    '신인천해수구취수구', '신풍리', '영월본부', '영월철도부지', '와산리', '용수리', 
    '위미2리', '이천D(백사면B)', '이천시 백사면A', '익산 다송리', '인천수산정수장', 
    '하동공설운동장', '하동변전소', '하동보건소', '하동정수장', '하동하수처리장', 
    '행원소수력', '화촌주민참여형', '부산본부', '삼척소내','하동본부'
]

def get_location_info(place_name):
    """카카오맵 API로 장소 정보 가져오기"""
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "query": place_name,
        "size": 1
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            return {
                'place_name': place_name,
                'status': 'API_ERROR',
                'error_message': f'HTTP {response.status_code}'
            }
        
        data = response.json()
        
        if 'documents' in data and len(data['documents']) > 0:
            doc = data['documents'][0]
            return {
                'place_name': place_name,
                'status': 'SUCCESS',
                'address': doc.get('address_name', ''),
                'road_address': doc.get('road_address_name', ''),
                'category': doc.get('category_name', ''),
                'phone': doc.get('phone', ''),
                'x': doc.get('x', ''),  # 경도
                'y': doc.get('y', ''),  # 위도
                'place_url': doc.get('place_url', '')
            }
        else:
            return {
                'place_name': place_name,
                'status': 'NO_RESULT',
                'address': '',
                'road_address': '',
                'category': '',
                'phone': '',
                'x': '',
                'y': '',
                'place_url': ''
            }
            
    except Exception as e:
        return {
            'place_name': place_name,
            'status': 'ERROR',
            'error_message': str(e),
            'address': '',
            'road_address': '',
            'category': '',
            'phone': '',
            'x': '',
            'y': '',
            'place_url': ''
        }

def extract_city_district(address):
    """주소에서 시/도, 구/군 정보 추출"""
    if not address:
        return '', '', ''
    
    # 시/도 추출
    city_pattern = r'([가-힣]+[시도])'
    city_match = re.search(city_pattern, address)
    city = city_match.group(1) if city_match else ''
    
    # 구/군 추출
    district_pattern = r'([가-힣]+[구군])'
    district_match = re.search(district_pattern, address)
    district = district_match.group(1) if district_match else ''
    
    # 동/읍/면 추출
    dong_pattern = r'([가-힣]+[동읍면])'
    dong_match = re.search(dong_pattern, address)
    dong = dong_match.group(1) if dong_match else ''
    
    return city, district, dong

def create_power_plant_dataframe():
    """발전소 위치 정보를 수집하여 DataFrame 생성"""
    print("태양광 발전소 위치 정보 수집을 시작합니다...")
    print(f"총 {len(power_plants)}개 발전소 처리 예정")
    print("=" * 50)
    
    results = []
    
    for i, plant in enumerate(power_plants, 1):
        print(f"[{i}/{len(power_plants)}] 검색 중: {plant}")
        
        # API 호출
        location_info = get_location_info(plant)
        
        # 주소 정보 추출
        if location_info['status'] == 'SUCCESS':
            city, district, dong = extract_city_district(location_info['address'])
            location_info['city'] = city
            location_info['district'] = district
            location_info['dong'] = dong
            
            print(f"  → 성공: {city} {district} {dong}")
        else:
            location_info['city'] = ''
            location_info['district'] = ''
            location_info['dong'] = ''
            print(f"  → 실패: {location_info.get('status', 'UNKNOWN')}")
        
        results.append(location_info)
        
        # API 호출 제한을 위한 대기
        time.sleep(0.1)
    
    # DataFrame 생성
    df = pd.DataFrame(results)
    
    # 컬럼 순서 정리
    columns_order = [
        'place_name', 'status', 'city', 'district', 'dong',
        'address', 'road_address', 'category', 'phone', 
        'x', 'y', 'place_url'
    ]
    
    # 에러 컬럼이 있는 경우 추가
    if 'error_message' in df.columns:
        columns_order.append('error_message')
    
    df = df[columns_order]
    
    print("=" * 50)
    print("수집 완료!")
    print(f"성공: {len(df[df['status'] == 'SUCCESS'])}개")
    print(f"실패: {len(df[df['status'] != 'SUCCESS'])}개")
    
    return df

def save_results(df, filename=None):
    """결과를 CSV 파일로 저장"""
    if filename is None:
        filename = f"solar_plant_locations.csv"

    df.to_csv(os.path.join(save_path, filename), index=False, encoding='utf-8-sig')
    print(f"결과가 '{filename}'에 저장되었습니다.")
    
    return filename

def print_summary(df):
    """결과 요약 출력"""
    print("\n=== 수집 결과 요약 ===")
    print(f"총 발전소 수: {len(df)}")
    print(f"성공: {len(df[df['status'] == 'SUCCESS'])}")
    print(f"검색결과 없음: {len(df[df['status'] == 'NO_RESULT'])}")
    print(f"API 오류: {len(df[df['status'] == 'API_ERROR'])}")
    print(f"기타 오류: {len(df[df['status'] == 'ERROR'])}")
    
    print("\n=== 지역별 분포 ===")
    successful_df = df[df['status'] == 'SUCCESS']
    city_counts = successful_df['city'].value_counts()
    for city, count in city_counts.items():
        print(f"{city}: {count}개")
    
    print("\n=== 구/군별 분포 ===")
    district_counts = successful_df['district'].value_counts()
    for district, count in district_counts.items():
        if district:  # 빈 값이 아닌 경우만
            print(f"{district}: {count}개")
    
    print("\n=== 검색 실패한 발전소 ===")
    failed_df = df[df['status'] != 'SUCCESS']
    if len(failed_df) > 0:
        for _, row in failed_df.iterrows():
            print(f"- {row['place_name']}: {row['status']}")
    else:
        print("모든 발전소 검색 성공!")

# 메인 실행 함수
def main():
    """메인 실행 함수"""
    # 데이터 수집
    df = create_power_plant_dataframe()
    
    # 결과 요약 출력
    print_summary(df)
    
    # 결과 저장
    filename = save_results(df)
    
    # 성공한 결과만 별도로 저장
    successful_df = df[df['status'] == 'SUCCESS']
    if len(successful_df) > 0:
        success_filename = filename.replace('.csv', '_success_only.csv')
        successful_df.to_csv(success_filename, index=False, encoding='utf-8-sig')
        print(f"성공한 결과만 '{success_filename}'에 저장되었습니다.")
    
    print("\n=== 처리 완료 ===")
    print("DataFrame 변수명: df")
    print("성공한 데이터만: successful_df")
    
    return df, successful_df

# 실행
if __name__ == "__main__":
    df, successful_df = main()
    
    # 결과 미리보기
    print("\n=== 결과 미리보기 ===")
    print(successful_df[['place_name', 'city', 'district', 'dong', 'address']].head(10))

/Users/soomin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


태양광 발전소 위치 정보 수집을 시작합니다...
총 37개 발전소 처리 예정
[1/37] 검색 중: 감우리
  → 성공:  음성군 음성읍
[2/37] 검색 중: 남제주소내
  → 실패: NO_RESULT
[3/37] 검색 중: 무릉리
  → 성공: 강원특별자치도 정선군 남면
[4/37] 검색 중: 부산복합자재창고
  → 실패: NO_RESULT
[5/37] 검색 중: 부산수처리장
  → 실패: NO_RESULT
[6/37] 검색 중: 부산신항
  → 성공:  강서구 생곡동
[7/37] 검색 중: 부산역선상주차장
  → 성공:  동구 초량동
[8/37] 검색 중: 부산운동장
  → 성공:  서구 서대신동
[9/37] 검색 중: 세화리
  → 성공: 제주특별자치도  구좌읍
[10/37] 검색 중: 송당리
  → 성공: 제주특별자치도  구좌읍
[11/37] 검색 중: 신인천 1_2단계 주차장
  → 실패: NO_RESULT
[12/37] 검색 중: 신인천 북측부지
  → 실패: NO_RESULT
[13/37] 검색 중: 신인천 주차장
  → 성공:  중구 운서동
[14/37] 검색 중: 신인천본관주차장
  → 실패: NO_RESULT
[15/37] 검색 중: 신인천소내
  → 실패: NO_RESULT
[16/37] 검색 중: 신인천전망대
  → 실패: NO_RESULT
[17/37] 검색 중: 신인천해수구취수구
  → 실패: NO_RESULT
[18/37] 검색 중: 신풍리
  → 성공: 여수시  율촌면
[19/37] 검색 중: 영월본부
  → 성공: 강원특별자치도 영월군 북면
[20/37] 검색 중: 영월철도부지
  → 실패: NO_RESULT
[21/37] 검색 중: 와산리
  → 성공: 제주특별자치도  조천읍
[22/37] 검색 중: 용수리
  → 성공:  기장군 정관읍
[23/37] 검색 중: 위미2리
  → 성공: 제주특별자치도  남원읍
[24/37] 검색 중: 이천D(백사면B)
  → 성공: 이천시  백사면
[25/37] 검색 중: 이천시 백사면A
  →

### 2. 매핑 안되거나 부정확한 애들은 아래 정보들 참고해서 눈으로 확인하고 직접 매핑함

In [10]:
detail_location = successful_df[['place_name', 'city', 'district', 'dong','address']].copy()

# 복사본 생성
new_df = detail_location.copy()

# 부산본부를 2개로 증식
busan_bonbu = new_df[new_df['place_name'] == '부산본부'].iloc[0].to_dict()
busan_rows = []

for i in range(1, 3):  # 1~2까지
    new_row = busan_bonbu.copy()
    new_row['place_name'] = f'부산본부_{i}'
    busan_rows.append(new_row)

# 삼척소내를 4개로 증식 (1부터 시작)
samcheok = new_df[new_df['place_name'] == '삼척소내'].iloc[0].to_dict()
samcheok_rows = []

for i in range(1, 5):  # 1~4까지
    new_row = samcheok.copy()
    new_row['place_name'] = f'삼척소내_{i}'
    samcheok_rows.append(new_row)

# 하동본부를 6개로 증식
hadong = new_df[new_df['place_name'] == '하동본부'].iloc[0].to_dict()
hadong_rows = []

for i in range(1, 7):  # 1~6까지
    new_row = hadong.copy()
    new_row['place_name'] = f'하동본부_{i}'
    hadong_rows.append(new_row)

# 원본에서 해당 행들 제거
new_df = new_df[~new_df['place_name'].isin(['부산본부', '삼척소내', '하동본부'])]

# 새로운 행들 추가
additional_rows = pd.DataFrame(busan_rows + samcheok_rows + hadong_rows)
result_df = pd.concat([new_df, additional_rows], ignore_index=True)
result_df = result_df.rename(columns={'place_name':'name'})
result_df

,name,city,district,dong,address
0,감우리,,음성군,음성읍,충북 음성군 음성읍 감우리 산 68-1
1,무릉리,강원특별자치도,정선군,남면,강원특별자치도 정선군 남면 무릉리
2,부산신항,,강서구,생곡동,부산 강서구 생곡동 1594-8
3,부산역선상주차장,,동구,초량동,부산 동구 초량동 1187-1
4,부산운동장,,서구,서대신동,부산 서구 서대신동3가 210-1
5,세화리,제주특별자치도,,구좌읍,제주특별자치도 제주시 구좌읍 세화리 1-1
6,송당리,제주특별자치도,,구좌읍,제주특별자치도 제주시 구좌읍 송당리 2860
7,신인천 주차장,,중구,운서동,인천 중구 운서동 2850-3
8,신풍리,여수시,,율촌면,전남 여수시 율촌면 신풍리 861-1
9,영월본부,강원특별자치도,영월군,북면,강원특별자치도 영월군 북면 마차리 1125-4


In [16]:
meta = pd.read_csv('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/solar_energy/meta_data.csv')

location_info = pd.merge(result_df,meta[['name','location_name','광역시']], on='name', how='right')
location_info

,name,city,district,dong,address,location_name,광역시
0,감우리,,음성군,음성읍,충북 음성군 음성읍 감우리 산 68-1,충주,충청북도
1,남제주소내,NaN,NaN,NaN,NaN,제주,제주특별자치도
2,무릉리,강원특별자치도,정선군,남면,강원특별자치도 정선군 남면 무릉리,서귀포,제주특별자치도
3,부산복합자재창고,NaN,NaN,NaN,NaN,부산,부산
4,부산수처리장,NaN,NaN,NaN,NaN,부산,부산
5,부산신항,,강서구,생곡동,부산 강서구 생곡동 1594-8,부산,부산
6,부산역선상주차장,,동구,초량동,부산 동구 초량동 1187-1,부산,부산
7,부산운동장,,서구,서대신동,부산 서구 서대신동3가 210-1,부산,부산
8,세화리,제주특별자치도,,구좌읍,제주특별자치도 제주시 구좌읍 세화리 1-1,성산일출,제주특별자치도
9,송당리,제주특별자치도,,구좌읍,제주특별자치도 제주시 구좌읍 송당리 2860,제주,제주특별자치도


In [18]:
d = pd.read_parquet('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_충북.parquet')
d['지역'].unique()

array(['충북 괴산군', '충북 단양군', '충북 제천시', '충북 청주시', '충북 충주시', '충북 진천군',
       '충북 옥천군', '충북 음성군', '충북 보은군', '충북 영동군', '충북 증평군'], dtype=object)

In [17]:
air = pd.read_parquet('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_강원.parquet')
air['지역'].unique()

array(['강원 강릉시', '강원 고성군', '강원 동해시', '강원 삼척시', '강원 양구군', '강원 원주시',
       '강원 정선군', '강원 춘천시', '강원 횡성군', '강원 평창군', '강원 양양군', '강원 철원군',
       '강원 속초시', '강원 영월군', '강원 인제군', '강원 태백시', '강원 홍천군', '강원 화천군'],
      dtype=object)

In [20]:
path = '/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_제주.parquet'
a = pd.read_parquet(path)
a['지역'].unique()

array(['제주 서귀포시', '제주 제주시', '제주 서귀포', '제주 제주'], dtype=object)

In [21]:
path = '/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_부산.parquet'
a = pd.read_parquet(path)
a['지역'].unique()

array(['부산 강서구', '부산 금정구', '부산 기장군', '부산 남구', '부산 동구', '부산 동래구',
       '부산 부산진구', '부산 북구', '부산 사상구', '부산 사하구', '부산 서구', '부산 수영구',
       '부산 연제구', '부산 영도구', '부산 중구', '부산 해운대구'], dtype=object)

In [22]:
path = '/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_인천.parquet'
a = pd.read_parquet(path)
a['지역'].unique()

array(['인천 강화군', '인천 계양구', '인천 남구', '인천 남동구', '인천 동구', '인천 부평구', '인천 서구',
       '인천 연수구', '인천 옹진군', '인천 중구', '인천 미추홀구', '인천 서구_x000D_\n',
       '인천 연수구_x000D_\n'], dtype=object)

In [23]:
path = '/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_경기.parquet'
a = pd.read_parquet(path)
a['지역'].unique()

array(['경기 가평군', '경기 고양시', '경기 과천시', '경기 광명시', '경기 광주시', '경기 구리시',
       '경기 군포시', '경기 김포시', '경기 남양주시', '경기 동두천시', '경기 부천시', '경기 성남시',
       '경기 수원시', '경기 시흥시', '경기 안산시', '경기 안성시', '경기 안양시', '경기 양주시',
       '경기 양평군', '경기 여주시', '경기 연천군', '경기 오산시', '경기 용인시', '경기 의왕시',
       '경기 의정부시', '경기 이천시', '경기 파주시', '경기 평택시', '경기 포천시', '경기 하남시',
       '경기 화성시'], dtype=object)

In [24]:
path = '/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_전북.parquet'
a = pd.read_parquet(path)
a['지역'].unique()

array(['전북 고창군', '전북 군산시', '전북 김제시', '전북 남원시', '전북 부안군', '전북 익산시',
       '전북 임실군', '전북 전주시', '전북 정읍시', '전북 완주군', '전북 진안군', '전북 무주군',
       '전북 순창군', '전북 장수군'], dtype=object)

In [25]:
path = '/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/air_data/preprocessed_data/air_경남.parquet'
a = pd.read_parquet(path)
a['지역'].unique()

array(['경남 거제시', '경남 거창군', '경남 김해시', '경남 사천시', '경남 양산시', '경남 진주시',
       '경남 창원시', '경남 하동군', '경남 밀양시', '경남 통영시', '경남 고성군', '경남 남해군',
       '경남 함안군', '경남 함양군', '경남 산청군', '경남 의령군', '경남 창녕군', '경남 합천군'],
      dtype=object)

### 3. 최종본!

In [11]:
import pandas as pd

# 데이터 생성
data = [
    ["감우리", "충북", "충주시"],
    ["남제주소내", "제주", "제주시"],
    ["무릉리", "제주", "서귀포시"],
    ["부산복합자재창고", "부산", "중구"],
    ["부산수처리장", "부산", "중구"],
    ["부산신항", "부산", "강서구"],
    ["부산역선상주차장", "부산", "동구"],
    ["부산운동장", "부산", "서구"],
    ["세화리", "제주", "제주시"],
    ["송당리", "제주", "제주시"],
    ["신인천 1_2단계 주차장", "인천", "서구"],
    ["신인천 북측부지", "인천", "서구"],
    ["신인천 주차장", "인천", "서구"],
    ["신인천본관주차장", "인천", "서구"],
    ["신인천소내", "인천", "서구"],
    ["신인천전망대", "인천", "서구"],
    ["신인천해수구취수구", "인천", "서구"],
    ["신풍리", "제주", "제주시"],
    ["영월본부", "강원", "영월군"],
    ["영월철도부지", "강원", "영월군"],
    ["와산리", "제주", "제주시"],
    ["용수리", "제주", "제주시"],
    ["위미2리", "제주", "서귀포시"],
    ["이천D(백사면B)", "경기", "이천시"],
    ["이천시 백사면A", "경기", "이천시"],
    ["익산 다송리", "전북", "전주시"],
    ["인천수산정수장", "인천", "남동구"],
    ["하동공설운동장", "경남", "하동군"],
    ["하동변전소", "경남", "하동군"],
    ["하동보건소", "경남", "하동군"],
    ["하동정수장", "경남", "하동군"],
    ["하동하수처리장", "경남", "하동군"],
    ["행원소수력", "제주", "제주시"],
    ["화촌주민참여형", "경남", "하동군"],
    ["부산본부_1", "부산", "중구"],
    ["부산본부_2", "부산", "중구"],
    ["삼척소내_1", "강원", "동해시"],
    ["삼척소내_2", "강원", "동해시"],
    ["삼척소내_3", "강원", "동해시"],
    ["삼척소내_4", "강원", "동해시"],
    ["하동본부_1", "경남", "하동군"],
    ["하동본부_2", "경남", "하동군"],
    ["하동본부_3", "경남", "하동군"],
    ["하동본부_4", "경남", "하동군"],
    ["하동본부_5", "경남", "하동군"],
    ["하동본부_6", "경남", "하동군"]
]

# DataFrame 생성
df = pd.DataFrame(data, columns=["name", "city", "지역"])
df.to_csv('../../data/solar_energy/solar_plant_locations.csv', index=False)
df

,name,city,지역
0,감우리,충북,충주시
1,남제주소내,제주,제주시
2,무릉리,제주,서귀포시
3,부산복합자재창고,부산,중구
4,부산수처리장,부산,중구
5,부산신항,부산,강서구
6,부산역선상주차장,부산,동구
7,부산운동장,부산,서구
8,세화리,제주,제주시
9,송당리,제주,제주시


### solar_plant_locations 정보를 Solar energy meta data에 추가하기

In [13]:
import pandas as pd

solar = pd.read_csv('../../data/solar_energy/solar_plant_locations.csv')
meta_data = pd.read_csv('../../data/solar_energy/meta_data.csv')
solar

,name,city,지역
0,감우리,충북,충주시
1,남제주소내,제주,제주시
2,무릉리,제주,서귀포시
3,부산복합자재창고,부산,중구
4,부산수처리장,부산,중구
5,부산신항,부산,강서구
6,부산역선상주차장,부산,동구
7,부산운동장,부산,서구
8,세화리,제주,제주시
9,송당리,제주,제주시


In [14]:
meta_data['city'] = solar['city']
meta_data['district'] = solar['지역']
meta_data

,name,발전소명,구분,용량(MW),준공년도,비 고,location_name,광역시,city,district
0,감우리,감우리,자체,0.5540,2021,RPS,충주,충청북도,충북,충주시
1,남제주소내,남제주소내태양광,자체,0.1960,2012,RPS,제주,제주특별자치도,제주,제주시
2,무릉리,무릉리,자체,0.8090,2021,RPS,서귀포,제주특별자치도,제주,서귀포시
3,부산복합자재창고,부산자재창고태양광,자체,0.1870,2012,RPS,부산,부산,부산,중구
4,부산수처리장,부산 수처리장 태양광,자체,0.1110,2017,RPS,부산,부산,부산,중구
5,부산신항,부산신항태양광,자체,0.1150,2011,RPA,부산,부산,부산,강서구
6,부산역선상주차장,부산역 선상 주차장,자체,0.8344,2020,RPS,부산,부산,부산,동구
7,부산운동장,부산 운동장부지 태양광,자체,1.4000,2017,RPS,부산,부산,부산,서구
8,세화리,세화리,자체,0.9990,2021,RPS,성산일출,제주특별자치도,제주,제주시
9,송당리,송당리 태양광,자체,0.9970,2019,RPS,제주,제주특별자치도,제주,제주시


In [15]:
meta_data.to_csv('../../data/solar_energy/meta_data.csv',index=False)